In [3]:
import sys
print(sys.executable)
print(sys.version)


c:\Users\ftc\AppData\Local\Programs\Python\Python310\python.exe
3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]


In [4]:
!pip install google-generativeai


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: C:\Python313\python.exe -m pip install --upgrade pip


In [2]:
!python -m pip install google-generativeai  

In [4]:
from dotenv import load_dotenv
load_dotenv()
import os
# os.environ["GOOGLE_API_KEY"] 

In [6]:
from langchain_google_genai import GoogleGenerativeAI

In [ ]:
import google.generativeai as genai


# api_key= "Your own key here"

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

llm = GoogleGenerativeAI(
      model="gemini-1.5-flash",
      google_api_key=os.environ["GOOGLE_API_KEY"],
      temperature=0.2,
  )


In [8]:
pip install InstructorEmbedding

Note: you may need to restart the kernel to use updated packages.


In [9]:
 pip install torch 

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install sentence-transformers


  Using cached sentence_transformers-5.1.0-py3-none-any.whl.metadata (16 kB)
  Using cached transformers-4.55.2-py3-none-any.whl.metadata (41 kB)
  Using cached huggingface_hub-0.34.4-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached sentence_transformers-5.1.0-py3-none-any.whl (483 kB)
Using cached transformers-4.55.2-py3-none-any.whl (11.3 MB)
Using cached huggingface_hub-0.34.4-py3-none-any.whl (561 kB)
Using cached tokenizers-0.21.4-cp39-abi3-win_amd64.whl (2.5 MB)

   ---------------------------------------- 0/4 [huggingface-hub]
   ---------------------------------------- 0/4 [huggingface-hub]
   ---------------------------------------- 0/4 [huggingface-hub]
   ---------------------------------------- 0/4 [huggingface-hub]
   ---------------------------------------- 0/4 [huggingface-hub]
   ---------------------------------------- 0/4 [huggingface-hub]
   ---------------------------------------- 0/4 [hugging

In [10]:
pip install InstructorEmbedding

Note: you may need to restart the kernel to use updated packages.


In [11]:

from langchain.document_loaders.csv_loader import CSVLoader

from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", 
    
)

database_path = "faiss_index_vectordb"

def create_vector_db():
    loader = CSVLoader(file_path="D:/CodeBasics_QandA/codebasics_faqs.csv",source_column='prompt')
    data = loader.load()
    # print(data)
    vectordb= FAISS.from_documents(documents=data, embedding=instructor_embeddings)
    vectordb.save_local(database_path)



create_vector_db()



C:\Users\ftc\AppData\Local\Temp\ipykernel_40192\1071870846.py:5: LangChainDeprecationWarning: The class `HuggingFaceInstructEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  instructor_embeddings = HuggingFaceInstructEmbeddings(
No sentence-transformers model found with name hkunlp/instructor-large. Creating a new one with mean pooling.
`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


In [12]:

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
def get_qa_chain():
    # load vectordb from loacal folder created
    vectordb = FAISS.load_local(database_path, instructor_embeddings, allow_dangerous_deserialization=True)
    retriever= vectordb.as_retriever(score_threshold=0.5)
    # rdocs= retriever.get_relevant_documents("what are the chances of job placement?")
    # rdocs

  

    prompt_template = """You are a helpful assistant. Given the following context and question, generate an answer based on this conext only, In the answer, try to provide as much text possible from the response "section" in the source document context without making it yourself. if the answer is not found in the context, kindly state "I don't know". Do not try to make up an answer. 

    CONTEXT: {context}
    QUESTION: {question}

    """

    PROMPT= PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    
    chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        input_key="query",
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )
    return chain

chain=get_qa_chain()
chain("WHat kind of courses do you offer?")



C:\Users\ftc\AppData\Local\Temp\ipykernel_40192\2978705946.py:36: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain("WHat kind of courses do you offer?")
`SentenceTransformer._target_device` has been deprecated, please use `SentenceTransformer.device` instead.


{'query': 'WHat kind of courses do you offer?',
 'result': "I don't know.\n",
 'source_documents': [Document(id='f957147d-4a4c-440a-a886-bc68625f8dd0', metadata={'source': 'Can I add this course to my resume? If Yes, how?', 'row': 34}, page_content='prompt: Can I add this course to my resume? If Yes, how?\nresponse: Absolutely, we have a section in this course explaining how you can add the learnings from this course in your resume that will appeal to the hiring team.'),
  Document(id='82ccce3d-0f10-4648-8c86-de3319a6683c', metadata={'source': 'Do you provide any job assistance?', 'row': 11}, page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.'),
  Document(id='1b6bc909-d39f-4dde-9371-75f7506ced9d', metadata={'source': 'I use tableau, can I take this course?', 'row': 28}, page_content='p